# Stoneburner, Kurt
- ## DSC 550 - Week 09/10

In [ ]:
import os
import sys
# //*** Imports and Load Data
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json

#//*** Use the whole window in the IPYNB editor
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#//*** Maximize columns and rows displayed by pandas
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)


### 2. Neural Network Classifier with Keras###
Using the multi-label classifier dataset from earlier exercises (categorized-comments.jsonl in the reddit folder), fit a neural network classifier using Keras. Use the code found in chapter 12 of the Applied Text Analysis with Python book as a guideline. Report the accuracy, precision, recall, F1-score, and confusion matrix.

In [ ]:
#https://oindrilasen.com/2021/02/how-to-install-and-import-keras-in-anaconda-jupyter-notebooks/

from keras.layers import Dense
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier


from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [ ]:
df = pd.read_pickle("z_wk09_categorized_comments_processed.zip")

df['processed'] = df['lema_stem_tokens'].apply(lambda word_list: ' '.join(word_list)) 

#//*** Convert categorical string to categorical int
#//*** Only run once to prevent iPython issues
if (df.dtypes['cat'] == object):
    cat_dict = dict(tuple(enumerate(df['cat'].unique())))
    #//*** Build sexcat Categorical column
    df['intcat'] = df['cat'].copy()
    
    #//*** replace values using the sex_dict dictionary
    for key,value in cat_dict.items():
        df['intcat'] = df['intcat'].replace(value,key)

In [ ]:
#//*** Reference Code: Applied Text Analysys with Python p282.



In [ ]:


def build_network():
    """
    Create a function that returns a compiled neural network
    """
    nn = Sequential()
    nn.add(Dense(100, activation='relu', input_shape=(100,)))
    
    nn.add(Dense(150, activation='relu'))
    nn.add(Dense(3, activation='softmax'))
    nn.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    return nn


#//*** Theoretically, this number should be return as the second value of the tfidf sparse matrix
N_FEATURES = 100

#//*** N_CLASSES the number of categories to solve for
N_CLASSES = 3
"""
pipeline = Pipeline([
    ('vect', TfidfVectorizer(max_features=10000)),
    ('nn', KerasClassifier(build_fn=build_network, 
                           epochs=200,
                           batch_size=120
                          ))
    
])
"""
#//*** X is Post Processed Data to evaluate
data_model_x = df['processed']
#//*** Initialize the Vectorizer

tfidf = TfidfVectorizer(max_features=N_FEATURES)

#print("Vectorizing")
#//*** Build the feature matrix, which is a weighted sparse matrix
#data_model_x  = tfidf.fit_transform(df['processed'])

data_model_y = df['intcat']

test_size=.5
test_size=.98
#continuous scoring model
#scoring = 'r2_score'
print("Splitting")

# split the data randomly into test/train sets
x_train, x_test, y_train, y_test = train_test_split(data_model_x, data_model_y, test_size =test_size)
print(x_train)
#x_train_2 = tfidf.fit_transform(x_train).toarray()
#print(x_train_2)
#x_test_2 = tfidf.fit_transform(x_test)
#y_train_2 = tfidf.fit_transform(y_train).toarray()
pipeline = Pipeline([
    ('vect', TfidfVectorizer(max_features=N_FEATURES)),
    ('nn', KerasClassifier(build_fn=build_network, 
                           epochs=200,
                           batch_size=120
                          ))
    
])
#scores = cross_val_score(pipeline, data_model_x,data_model_y,cv=3,scoring='accuracy', n_jobs=-1)
#print(scores)
pipeline.fit(x_train,y_train)

In [ ]:
import tensorflow as tf

def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.sparse.reorder(tf.SparseTensor(indices, coo.data, coo.shape))

def build_network():
    """
    Create a function that returns a compiled neural network
    """
    nn = Sequential()
    nn.add(Dense(N_FEATURES, activation='relu', input_shape=(N_FEATURES,)))
    
    nn.add(Dense(N_FEATURES*.25, activation='relu'))
    nn.add(Dense(N_CLASSES, activation='softmax'))
    nn.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    return nn


#//*** Theoretically, this number should be return as the second value of the tfidf sparse matrix
N_FEATURES = 5000

#//*** N_CLASSES the number of categories to solve for
N_CLASSES = len(df['intcat'].unique())

#//*** X is Post Processed Data to evaluate
data_model_x = df['processed']
#//*** Initialize the Vectorizer

tfidf = TfidfVectorizer(max_features=N_FEATURES)

data_model_y = df['intcat']

test_size=.5
test_size=.8
#continuous scoring model
#scoring = 'r2_score'
print("Splitting")

#data_model_x = tfidf.fit_transform(data_model_x)

# split the data randomly into test/train sets
x_train, x_test, y_train, y_test = train_test_split(data_model_x, data_model_y, test_size =test_size)


vectorizer = TfidfVectorizer(max_features=N_FEATURES)

 

x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

x_train = convert_sparse_matrix_to_sparse_tensor(x_train)
x_test = convert_sparse_matrix_to_sparse_tensor(x_test)

pipeline = Pipeline([
#  ('norm', TextNormalizer()),
#  ('vect', TfidfVectorizer(max_features=N_FEATURES)),
 ('nn', KerasClassifier(build_fn=build_network,
 epochs=100,
batch_size=128))
 ])


pipeline.fit(x_train,y_train)


In [ ]:
y_predicted = pipeline.predict(x_test)

from sklearn.metrics import f1_score, precision_score, accuracy_score, recall_score, confusion_matrix

#You'll get a warning but just ignore it

 


print(f1_score(y_test, y_predicted, average = 'micro'))
print(precision_score(y_test, y_predicted, average = 'micro'))
print(accuracy_score(y_test, y_predicted))
print(recall_score(y_test, y_predicted, average = 'micro'))
print(confusion_matrix(y_test, y_predicted))

In [ ]:
from keras.layers import Dense
from keras.models import Sequential

 


N_FEATURES = 5000
N_CLASSES = 3
def build_network():
    """
    Create a function that returns a compiled neural network
    """
    nn = Sequential()
    nn.add(Dense(500, activation='relu', input_shape=(N_FEATURES,)))
    nn.add(Dense(150, activation='relu'))
    nn.add(Dense(N_CLASSES, activation='softmax'))
    nn.compile(
         loss='categorical_crossentropy',
         optimizer='adam',
         metrics=['accuracy']
    )
    return nn

 


def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.sparse.reorder(tf.SparseTensor(indices, coo.data, coo.shape))

 


import tensorflow as tf

 

vectorizer = TfidfVectorizer(max_features=N_FEATURES)

 

X_train = vectorizer.fit_transform(df["processed"])
X_test = vectorizer.transform(test["processed_text"])
X_train = convert_sparse_matrix_to_sparse_tensor(X_train)
X_test = convert_sparse_matrix_to_sparse_tensor(X_test)

 

 

 

from sklearn.pipeline import Pipeline
# from transformer import TextNormalizer
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
pipeline = Pipeline([
#  ('norm', TextNormalizer()),
#  ('vect', TfidfVectorizer(max_features=N_FEATURES)),
 ('nn', KerasClassifier(build_fn=build_network,
 epochs=200,
batch_size=128))
 ])

 


def train_model(X, y, model, saveto=None, cv=12): 
    """
    Trains model from corpus at specified path and fits on full data.
    If a saveto dictionary is specified, writes Keras and Sklearn
    pipeline components to disk separately. Returns the scores.
    """

 

    model.fit(X, y)
    if saveto: 
        model.steps[-1][1].model.save(saveto['keras_model']) 
        model.steps.pop(-1)
        joblib.dump(model, saveto['sklearn_pipe'])
    return model

 

    y_train = train["cat"]
y_test = test["cat"]

 

 

TModel =train_model(X_train, y_train, model = pipeline)

 

#scores being returned are the "loss scores" 

 

y_predicted = TModel.predict(X_test)

 

#You'll get a warning but just ignore it

 


f1_score(ytest, y_predicted, average = 'micro')
precision_score(ytest, y_predicted, average = 'micro')
accuracy_score(ytest, y_predicted)
recall_score(ytest, y_predicted, average = 'micro')
confusion_matrix(ytest, y_predicted)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

#//*** Initialize the Vectorizer
tfidf = TfidfVectorizer()


#//*** Build the feature matrix, which is a weighted sparse matrix
bwarg = tfidf.fit_transform(df['processed'][:10000])



In [ ]:
print(bwarg.shape)

### 3. Classifying Images ###
In chapter 20 of the Machine Learning with Python Cookbook, implement the code found in section 20.15 classify MSINT images using a convolutional neural network. Report the accuracy of your results.


In [1]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K




In [3]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

# Set that the color channel value will be first
K.set_image_data_format("channels_first")
#K.set_image_data_format("channels_last")

# Set seed
np.random.seed(0)

# Set image information
channels = 1
height = 28
width = 28

# Load data and target from MNIST data
(data_train, target_train), (data_test, target_test) = mnist.load_data()

# Reshape training image data into features
data_train = data_train.reshape(data_train.shape[0], channels, height, width)

# Reshape test image data into features
data_test = data_test.reshape(data_test.shape[0], channels, height, width)

# Rescale pixel intensity to between 0 and 1
features_train = data_train / 255
features_test = data_test / 255

# One-hot encode target
target_train = np_utils.to_categorical(target_train)
target_test = np_utils.to_categorical(target_test)
number_of_classes = target_test.shape[1]

# Start neural network
network = Sequential()

# Add convolutional layer with 64 filters, a 5x5 window, and ReLU activation function
network.add(Conv2D(filters=64,
                   kernel_size=(5, 5),
                   #kernel_size=(3, 3),
                   input_shape=(channels, width, height),
                   activation='relu'))

# Add max pooling layer with a 2x2 window
network.add(MaxPooling2D(pool_size=(2, 2)))


# Add dropout layer
network.add(Dropout(0.5))

# Add layer to flatten input
network.add(Flatten())

# # Add fully connected layer of 128 units with a ReLU activation function
network.add(Dense(128, activation="relu"))

# Add dropout layer
network.add(Dropout(0.5))

# Add fully connected layer with a softmax activation function
network.add(Dense(number_of_classes, activation="softmax"))

# Compile neural network
network.compile(loss="categorical_crossentropy", # Cross-entropy
                optimizer="rmsprop", # Root Mean Square Propagation
                metrics=["accuracy"]) # Accuracy performance metric

# Train neural network
network.fit(features_train, # Features
            target_train, # Target
            epochs=10, # Number of epochs
            verbose=1, # Don't print description after each epoch
            batch_size=1000, # Number of observations per batch
            validation_data=(features_test, target_test)) # Data for evaluation

Epoch 1/10
60/60 [==============================] - 13s 216ms/step - loss: 1.0217 - accuracy: 0.6662 - val_loss: 0.1555 - val_accuracy: 0.9557
Epoch 2/10
60/60 [==============================] - 13s 211ms/step - loss: 0.2187 - accuracy: 0.9365 - val_loss: 0.0855 - val_accuracy: 0.9733
Epoch 3/10
60/60 [==============================] - 13s 220ms/step - loss: 0.1330 - accuracy: 0.9595 - val_loss: 0.0631 - val_accuracy: 0.9789
Epoch 4/10
60/60 [==============================] - 13s 225ms/step - loss: 0.1019 - accuracy: 0.9705 - val_loss: 0.0490 - val_accuracy: 0.9844
Epoch 5/10
60/60 [==============================] - 13s 224ms/step - loss: 0.0845 - accuracy: 0.9741 - val_loss: 0.0430 - val_accuracy: 0.9846
Epoch 6/10
60/60 [==============================] - 13s 224ms/step - loss: 0.0696 - accuracy: 0.9790 - val_loss: 0.0379 - val_accuracy: 0.9866
Epoch 7/10
60/60 [==============================] - 13s 223ms/step - loss: 0.0675 - accuracy: 0.9801 - val_loss: 0.0350 - val_accuracy: 0.9877

**Model Accuracy**

In [17]:
# https://keras.io/examples/vision/mnist_convnet/
score = network.evaluate(data_test, target_test, verbose=1)
print("Test loss:", score[0])
print("Test accuracy:", score[1])


313/313 [==============================] - 1s 3ms/step - loss: 6.8145 - accuracy: 0.9868
Test loss: 6.814472675323486
Test accuracy: 0.9868000149726868
